In [ ]:
#The autoencoder was trained on the following different hyperparameters
# learning_rate = 1e-4, 3e-3
# batch_size = 10,100
# num_epochs = 3,8

# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#FFCE30"><b><span style='color:#FFFFFF'>1 |</span></b> <b>INTRODUCTION</b></div>

👋 Welcome to "🧠Exploring EEG: A Beginner's Guide"! 

If you're fascinated by the wonders of the human brain and the intricate patterns of brainwaves, but find the world of Electroencephalography (EEG) analysis daunting, you're in the right place. 

This notebook is designed for beginners like me & you, aiming to demystify the complexities of EEG data and make your learning journey both enjoyable and informative.

### <b><span style='color:#FFCE30'> 1.1 |</span> Intention of the notebook</b>
In this notebook, we will embark on an exploratory journey into the realm of EEG data analysis. Our goal is to provide a clear, step-by-step guide to understanding and analyzing EEG signals, which are crucial in detecting and classifying brain activities, such as seizures. We aim to:

* Break down complex concepts into easily digestible sections.
* Illustrate each step with practical code examples.
* Reference public notebooks and discussions to enhance your learning experience.


### <b><span style='color:#FFCE30'> 1.2 |</span> Learning Objective</b>
By the end of this notebook, you will have a foundational understanding of:

* The basics of EEG signals and their significance in medical research and neurology.
* How to preprocess and analyze EEG data.
* Run through the basic code to build a machine learning model for EEG data classification.

# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#FFCE30"><b><span style='color:#FFFFFF'>2 |</span></b> <b>REFERENCE & ACKNOWLEDGEMENT</b></div>

This notebook wouldn't be possible without the valuable insights and contributions from the Kaggle community. I've leveraged several resources to compile the most effective learning path for us:

* https://www.kaggle.com/code/cdeotte/catboost-starter-lb-0-8
* https://www.kaggle.com/code/mvvppp/hms-eda-and-domain-journey
* https://www.kaggle.com/code/ksooklall/hms-banana-montage
* https://www.kaggle.com/code/mpwolke/seizures-classification-parquet


Feel free to explore these resources alongside this notebook to deepen your understanding.

# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#FFCE30"><b><span style='color:#FFFFFF'>3 |</span></b> <b>LOAD LIBARIES</b></div>

In [1]:
#How to integrate GPU into the notebook

In [2]:
import os
import pandas as pd, numpy as np
from glob import glob
import matplotlib.pyplot as plt
VER = 1

# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#FFCE30"><b><span style='color:#FFFFFF'>4 |</span></b> <b>INTRODUCTION TO EEG AND SEIZURE DETECTION</b></div>

<b><span style='color:#FFCE30'> 4.1 |</span> Electroencephalography (EEG) - The Window into Brain Activity</b>

* Electroencephalography, commonly known as EEG, is a non-invasive method used by medical professionals to record electrical activity in the brain. 
* This is done using electrodes placed along the scalp. 
* EEG is a crucial tool in diagnosing neurological disorders, especially epilepsy, which is characterized by recurrent seizures.

<img src="https://www.researchgate.net/profile/Sebastian-Nagel-4/publication/338423585/figure/fig1/AS:844668573073409@1578396089381/Sketch-of-how-to-record-an-Electroencephalogram-An-EEG-allows-measuring-the-electrical.png" alt="EEG" width="600" height="400">



In [3]:
# check the reading of one parquet for understanding

BASE_PATH = '/kaggle/input/hms-harmful-brain-activity-classification/'

df = pd.DataFrame({'path': glob(BASE_PATH + '**/*.parquet')})
df['test_type'] = df['path'].str.split('/').str.get(-2).str.split('_').str.get(-1)
df['id'] = df['path'].str.split('/').str.get(-1).str.split('.').str.get(0)

df_eeg = pd.read_parquet(BASE_PATH + 'train_eegs/1000913311.parquet')
df_eeg.head()

,Fp1,F3,C3,P3,F7,T3,T5,O1,Fz,Cz,Pz,Fp2,F4,C4,P4,F8,T4,T6,O2,EKG
0,-105.849998,-89.230003,-79.459999,-49.230000,-99.730003,-87.769997,-53.330002,-50.740002,-32.250000,-42.099998,-43.270000,-88.730003,-74.410004,-92.459999,-58.930000,-75.739998,-59.470001,8.210000,66.489998,1404.930054
1,-85.470001,-75.070000,-60.259998,-38.919998,-73.080002,-87.510002,-39.680000,-35.630001,-76.839996,-62.740002,-43.040001,-68.629997,-61.689999,-69.320000,-35.790001,-58.900002,-41.660000,196.190002,230.669998,3402.669922
2,8.840000,34.849998,56.430000,67.970001,48.099998,25.350000,80.250000,48.060001,6.720000,37.880001,61.000000,16.580000,55.060001,45.020000,70.529999,47.820000,72.029999,-67.180000,-171.309998,-3565.800049
3,-56.320000,-37.279999,-28.100000,-2.820000,-43.430000,-35.049999,3.910000,-12.660000,8.650000,3.830000,4.180000,-51.900002,-21.889999,-41.330002,-11.580000,-27.040001,-11.730000,-91.000000,-81.190002,-1280.930054
4,-110.139999,-104.519997,-96.879997,-70.250000,-111.660004,-114.430000,-71.830002,-61.919998,-76.150002,-79.779999,-67.480003,-99.029999,-93.610001,-104.410004,-70.070000,-89.250000,-77.260002,155.729996,264.850006,4325.370117


In [4]:
# Determine the number of channels
# Assuming each row is a time point and each column is a channel
n_channels = df_eeg.shape[1]
n_channels

20

* The headers in the dataset (Fp1, F3, C3, P3, F7, T3, T5, O1, Fz, Cz, Pz, Fp2, F4, C4, P4, F8, T4, T6, O2, EKG) are standard electrode placement labels used in electroencephalography (EEG). 
* These labels correspond to specific positions on the scalp where EEG electrodes are placed to record brain activity. 
* Here's a brief overview of what they represent:

1. **Fp1, Fp2:** Frontopolar electrodes, located on the forehead, left and right side.
2. **F3, F4:** Frontal electrodes, on the left and right side of the forehead.
3. **C3, C4:** Central electrodes, placed above the left and right hemispheres of the brain.
4. **P3, P4:** Parietal electrodes, located on the upper back portion of the head, left and right sides.
5. **O1, O2:** Occipital electrodes, positioned at the back of the head near the visual cortex.
6. **T3, T4, T5, T6:** Temporal electrodes, situated on the left and right sides of the head near the ears. They are often involved in monitoring auditory functions.
7. **F7, F8:** Frontal-temporal electrodes, located at the front of the temporal lobes.
8. **Fz, Cz, Pz:** Midline electrodes, located at the frontal (Fz), central (Cz), and parietal (Pz) positions on the midline of the head.
9. **EKG:** Electrocardiogram electrode, which records the heart’s electrical activity. It's not directly related to brain activity but can be important in some EEG analyses.


<img src="https://www.researchgate.net/profile/Danny-Plass-Oude-Bos/publication/237777779/figure/fig3/AS:669556259434497@1536646060035/10-20-system-of-electrode-placement.png" alt="10-20-system-of-electrode-placement" width="300" height="150">

<b><span style='color:#FFCE30'> 4.2 |</span> Seizures and Their Impact</b>
* Seizures are sudden, uncontrolled electrical disturbances in the brain that can cause changes in behavior, feelings, movements, and levels of consciousness. 
* Detecting and classifying seizures accurately is vital for appropriate treatment and care, especially in critically ill patients.

<b><span style='color:#FFCE30'> 4.3 |</span> The Challenge of Manual EEG Analysis</b>

* Traditionally, EEG data analysis relies on visual inspection by trained neurologists. 
* This process is not only time-consuming and labor-intensive but also prone to errors due to fatigue and subjective interpretation.

<img src="https://slideplayer.com/slide/12925171/78/images/2/Manual+Interpretation+of+EEGs.jpg" alt="Manual Interpretation of EEG" width="700" height="300">
Source: Automated Identification of Abnormal Adult EEG, S. López, G. Suarez, D. Jungreis, I. Obeid and J. Picone, Neural Engineering Data Consortium, Temple University


<b><span style='color:#FFCE30'> 4.4 |</span> The Role of Data Science in EEG Analysis</b>

* Automating EEG Interpretation
The advent of machine learning and data science offers an opportunity to automate the interpretation of EEG data. By developing algorithms that can detect and classify different patterns in EEG signals, we can aid neurologists in making faster, more accurate diagnoses.

* The Data Science Approach
Data scientists approach this challenge by first preprocessing the EEG data, which involves filtering out noise and extracting relevant features. Machine learning models are then trained on these features to distinguish between different types of brain activity.

<img src="https://www.researchgate.net/profile/Huiguang-He/publication/336336651/figure/fig1/AS:834361356197888@1575938657076/The-flow-chart-of-EEG-emotion-classification-with-similarity-learning-network.png" alt="flowchart for EEG classification" width="700" height="300">


<b><span style='color:#FFCE30'> 4.5 |</span> Understanding EEG Patterns</b>

In the realm of EEG analysis for seizure detection, certain patterns are of particular interest:

1. **Seizure (SZ):** Characterized by abnormal rhythmic activity, indicative of a seizure.
2. **Generalized Periodic Discharges (GPD):** Patterns that may be seen in various encephalopathies.
3. **Lateralized Periodic Discharges (LPD):** Often associated with focal brain lesions.
4. **Lateralized Rhythmic Delta Activity (LRDA):** Can be observed in focal brain dysfunction.
5. **Generalized Rhythmic Delta Activity (GRDA):** Typically related to diffuse brain dysfunction.
6. **"Other" Patterns:** Any other type of activity not falling into the above categories.

<b><span style='color:#FFCE30'> 4.6 |</span> Interpreting Complex EEG Data</b>

EEG data interpretation can be complex, especially in edge cases where expert neurologists may not agree on a classification. This is where machine learning models can particularly shine by providing an additional layer of analysis.

<img src="https://www.neurology.org/cms/10.1212/WNL.0000000000207127/asset/bd84c182-712c-41ab-8742-cecf9d49a322/assets/images/large/5ff2.jpg" alt="flowchart for EEG classification" width="700" height="300">

Source: Development of Expert-Level Classification of Seizures and Rhythmic and Periodic Patterns During EEG Interpretation https://www.neurology.org/doi/10.1212/WNL.0000000000207127


# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#FFCE30"><b><span style='color:#FFFFFF'>5 |</span></b> <b>LOAD TRAIN DATA</b></div>

In [5]:
df = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/train.csv')
TARGETS = df.columns[-6:]
print('Train shape:', df.shape )
print('Targets', list(TARGETS))
df.head()

Train shape: (106800, 15)
Targets ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']


,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,0,0,0,0,0
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,3,0,0,0,0,0
2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,3,0,0,0,0,0
3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,3,0,0,0,0,0
4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,3,0,0,0,0,0


# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#FFCE30"><b><span style='color:#FFFFFF'>6 |</span></b> <b>CREATE NON-OVERLAPPING EEG ID TRAIN DATA</b></div>

Following the notebook from Chris Deotte: https://www.kaggle.com/code/cdeotte/catboost-starter-lb-0-8,
Initial discussion found here https://www.kaggle.com/competitions/hms-harmful-brain-activity-classification/discussion/467021

We perform the following because:

* **Match Training Data with Test Data Format:** The competition states that the test data does not have multiple segments from the same eeg_id. To make the training data similar to the test data, we also use only one segment per eeg_id in the training data.

* **Remove Redundancies:** This approach ensures that the training data does not have overlapping or redundant information, which can lead to a more accurate and generalizable machine learning model.

* **Consistency in Data:** By standardizing how we handle the EEG segments in training, we ensure that our model learns from data that is consistent in format with the data it will be tested on.

* **Data Preparation for Machine Learning:** The normalization of target variables and inclusion of relevant features like patient_id and expert_consensus prepare the dataset for effective machine learning modeling.

In [6]:
# Creating a Unique EEG Segment per eeg_id:
# The code groups (groupby) the EEG data (df) by eeg_id. Each eeg_id represents a different EEG recording.
# It then picks the first spectrogram_id and the earliest (min) spectrogram_label_offset_seconds for each eeg_id. This helps in identifying the starting point of each EEG segment.
# The resulting DataFrame train has columns spec_id (first spectrogram_id) and min (earliest spectrogram_label_offset_seconds).
train = df.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
    {'spectrogram_id':'first','spectrogram_label_offset_seconds':'min'})
train.columns = ['spec_id','min']


# Finding the Latest Point in Each EEG Segment:
# The code again groups the data by eeg_id and finds the latest (max) spectrogram_label_offset_seconds for each segment.
# This max value is added to the train DataFrame, representing the end point of each EEG segment.
tmp = df.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
    {'spectrogram_label_offset_seconds':'max'})
train['max'] = tmp


tmp = df.groupby('eeg_id')[['patient_id']].agg('first') # The code adds the patient_id for each eeg_id to the train DataFrame. This links each EEG segment to a specific patient.
train['patient_id'] = tmp


tmp = df.groupby('eeg_id')[TARGETS].agg('sum') # The code sums up the target variable counts (like votes for seizure, LPD, etc.) for each eeg_id.
for t in TARGETS:
    train[t] = tmp[t].values
    
y_data = train[TARGETS].values # It then normalizes these counts so that they sum up to 1. This step converts the counts into probabilities, which is a common practice in classification tasks.
y_data = y_data / y_data.sum(axis=1,keepdims=True)
train[TARGETS] = y_data

tmp = df.groupby('eeg_id')[['expert_consensus']].agg('first') # For each eeg_id, the code includes the expert_consensus on the EEG segment's classification.
train['target'] = tmp

train = train.reset_index() # This makes eeg_id a regular column, making the DataFrame easier to work with.
print('Train non-overlapp eeg_id shape:', train.shape )
train.head()

Train non-overlapp eeg_id shape: (17089, 12)


,eeg_id,spec_id,min,max,patient_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,target
0,568657,789577333,0.0,16.0,20654,0.0,0.000000,0.25,0.000000,0.166667,0.583333,Other
1,582999,1552638400,0.0,38.0,20230,0.0,0.857143,0.00,0.071429,0.000000,0.071429,LPD
2,642382,14960202,1008.0,1032.0,5955,0.0,0.000000,0.00,0.000000,0.000000,1.000000,Other
3,751790,618728447,908.0,908.0,38549,0.0,0.000000,1.00,0.000000,0.000000,0.000000,GPD
4,778705,52296320,0.0,0.0,40955,0.0,0.000000,0.00,0.000000,0.000000,1.000000,Other


In [7]:
train[(train.spec_id == 1908433744) & (train['min'] // 1000 == 2)]

,eeg_id,spec_id,min,max,patient_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,target
564,147350182,1908433744,2615.0,2615.0,17408,0.0,0.0,1.0,0.0,0.0,0.0,GPD
16281,4084934272,1908433744,2063.0,2063.0,17408,0.0,0.0,1.0,0.0,0.0,0.0,GPD
16954,4255016832,1908433744,2783.0,2783.0,17408,0.0,0.0,1.0,0.0,0.0,0.0,GPD
17060,4285210475,1908433744,2845.0,2845.0,17408,0.0,0.0,1.0,0.0,0.0,0.0,GPD


# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#FFCE30"><b><span style='color:#FFFFFF'>7 |</span></b> <b>FEATURE ENGINEERING</b></div>



<b><span style='color:#FFCE30'> 7.1 |</span> 10 min and 20 sec windows</b>

* The code belows efficiently reads spectrogram data, from a single combined file, based on the set variable. We relied on the dataset by Chris Deotte to save time. https://www.kaggle.com/datasets/cdeotte/brain-spectrograms
* It then performs feature engineering by calculating mean and minimum values over two different time windows for each frequency in the spectrogram.
It produce produces in 1600 features (400 features × 4 calculations) for each EEG ID.
* The new features are intended to help the model better understand and classify the EEG data.
* This approach is designed to enhance the model's performance by providing it with more detailed information derived from the spectrogram data.

In [8]:
READ_SPEC_FILES = False # If READ_SPEC_FILES is False, the code reads the combined file instead of individual files.
FEATURE_ENGINEER = True
READ_EEG_SPEC_FILES = False

In [9]:
%%time
# READ ALL SPECTROGRAMS
PATH = '/kaggle/input/hms-harmful-brain-activity-classification/train_spectrograms/'
files = os.listdir(PATH)
print(f'There are {len(files)} spectrogram parquets')

if READ_SPEC_FILES:    
    spectrograms = {}
    for i,f in enumerate(files):
        if i%100==0: print(i,', ',end='')
        tmp = pd.read_parquet(f'{PATH}{f}')
        name = int(f.split('.')[0])
        spectrograms[name] = tmp.iloc[:,1:].values
else:
    spectrograms = np.load('/kaggle/input/brain-spectrograms/specs.npy',allow_pickle=True).item()

There are 11138 spectrogram parquets
CPU times: user 2.88 s, sys: 6.14 s, total: 9.01 s
Wall time: 57.9 s


# Autoencoder Definition

In [10]:

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
 
# Define the autoencoder architecture
class Autoencoder2D(nn.Module):
    def __init__(self):
        super(Autoencoder2D, self).__init__()
        #Where is this input layer?
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding= 1), #For detecting features
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 8, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(8, 16, 
                               kernel_size=3, 
                               stride=2,
                               padding=1, 
                               output_padding=1), #For creating features
            nn.ReLU(), 
            nn.ConvTranspose2d(16, 32, 
                               kernel_size=3, 
                               stride=2,
                               padding=1, 
                               output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 64, 
                               kernel_size=3, 
                               stride=2,
                               padding=1, 
                               output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 1, 
                               kernel_size=3, 
                               stride=2,
                               padding=1, 
                               output_padding=1),
            nn.Sigmoid()
        )
    def forward(self, x):
        #print(x.shape) #-> [1, 10, 400] (Only for 20 second)
        #x = x.reshape(1,batch_size,x.shape[1],x.shape[2])
        x = x.reshape((x.shape[0], 1, x.shape[1], x.shape[2]))
        x = self.encoder(x)
        x = self.decoder(x)
        #print(x.shape) #-> [1, 8, 400]
        x = F.pad(x, (0, 0, 0, 12), mode='constant', value=0)
        return x

In [11]:
#Potentially decrease size of encoded data 
#Convolutional 2D takes in only 2d data, we have 3d data here!
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn.functional as F
 
# Define the autoencoder architecture
class Autoencoder20second2D(nn.Module):
    def __init__(self):
        super(Autoencoder20second2D, self).__init__()
        #What to replace the batch size (in channels) parameter with?
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding= 1), #For detecting features
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 8, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(8, 32, 
                               kernel_size=3, 
                               stride=2,
                               padding=1, 
                               output_padding=1), #For creating features
            nn.ReLU(), 
            nn.ConvTranspose2d(32, 1, 
                               kernel_size=3, 
                               stride=2,
                               padding=1, 
                               output_padding=1),
            nn.Sigmoid(),
        )
    def forward(self, x):
        x = x.reshape((x.shape[0], 1, x.shape[1], x.shape[2]))
        #x = x.reshape(1,batch_size,x.shape[1],x.shape[2])
        x = self.encoder(x)
        x = self.decoder(x)
        x = F.pad(x, (0, 0, 1, 1), mode='constant', value=0) #Single Dimension Padding! -> Should this be in the input
        return x

In [12]:
#3D Convolutional Autoencoder

# Autoencoder Feature Engineering - Spectrogram Level

###
CNN Autoencoder
###

In [13]:
%time
# ENGINEER FEATURES
import warnings
warnings.filterwarnings('ignore')
SPEC_FREQS = len(pd.read_parquet(f'{PATH}1000086677.parquet').columns[1:])
print(f"Num Frequencies: {SPEC_FREQS}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu') # delete when issue resolved
print("Using: ", device)
"""
Define 10min feature autoencoder
"""
#model_10min = ConvAutoencoder(SPEC_FREQS, 400, numFeatures)
model_10min = Autoencoder2D()
model_10min = model_10min.to(device)
loss_function_10min = torch.nn.MSELoss()
optimizer_10min = torch.optim.Adam(model_10min.parameters(),
                            lr = learning_rate,
                            )

"""
Define 20sec feature autoencoder
"""
#model_20sec = ConvAutoencoder(SPEC_FREQS, 400, numFeatures)
model_20sec = Autoencoder20second2D()
model_20sec = model_20sec.to(device)
loss_function_20sec = torch.nn.MSELoss()
optimizer_20sec = torch.optim.Adam(model_20sec.parameters(),
                            lr = learning_rate,
                            )

CPU times: user 2 µs, sys: 2 µs, total: 4 µs
Wall time: 6.68 µs
Num Frequencies: 400
Using:  cuda


In [14]:
# print(batch_size)
# print(len(train) // batch_size + 1)
# len(train)

In [15]:
""" For 2D Convolutional Autoencoder 
Does stochastic gradient descent on the entire dataset
"""
%timeit
from torch.utils.data import DataLoader, TensorDataset
from sklearn.impute import SimpleImputer

def extract_raw_values(spectrogram_id, r, offset, end):
    return spectrograms[spectrogram_id][r+offset:r+end,:]

# Create a SimpleImputer instance
nan_imputer = SimpleImputer(strategy='mean')

print(f"Training Autoencoder on {len(train)} datapoints with batch size {batch_size}")
for epoch in range(num_epochs): 
    num_batches = len(train) // batch_size + 1
    epoch_loss_10min = 0.0
    epoch_loss_20sec = 0.0
    print(f"Batches {num_batches}:", end=' ')
    
    for i in range(num_batches):        
        
        row_start = i * batch_size #Currently, it is i
        row_end = min((i + 1) * batch_size, len(train)) #Currently, it is (i+1)
        
        input_10min_list = []
        input_20sec_list = []
        
        for k in range(row_start, row_end):
            row = train.iloc[k]
            r = int((row['min'] + row['max']) // 4)

            # get raw spectrogram values
            raw_values_10min = extract_raw_values(row.spec_id, r, 0, 300)
            raw_values_20sec = extract_raw_values(row.spec_id, r, 145, 155)

            # Replace infinite values
            raw_values_10min = np.where(np.isfinite(raw_values_10min), raw_values_10min, np.nan)
            raw_values_20sec = np.where(np.isfinite(raw_values_20sec), raw_values_20sec, np.nan)            
            
            # Use SimpleImputer to handle NaN values
            raw_values_10min = nan_imputer.fit_transform(raw_values_10min)
            raw_values_20sec = nan_imputer.fit_transform(raw_values_20sec)
            
            
            #Convert to torch tensors and append to the lists
            if len(raw_values_10min.flatten()) == 120000:
                # normalize
                # original_shape=raw_values_10min.shape
                # raw_values_10min = np.log(raw_values_10min.flatten() + log_noise)
                #raw_values_10min = raw_values_10min.flatten()
                #raw_values_10min = np.log(raw_values_10min + log_noise)
                normalized_values_10min = (raw_values_10min - raw_values_10min.min()) / (raw_values_10min.max() - raw_values_10min.min())
                #normalized_values_10min = normalized_values_10min.reshape(original_shape)
                input_10min_list.append(normalized_values_10min)
            if len(raw_values_20sec.flatten()) == 4000:
                # normalize
                #original_shape=raw_values_20sec.shape
                #raw_values_20sec = np.log(raw_values_20sec.flatten()  + log_noise)
                #raw_values_20sec = raw_values_20sec.flatten()
                #raw_values_20sec = np.log(raw_values_20sec + log_noise)
                normalized_values_20sec = (raw_values_20sec - raw_values_20sec.min()) / (raw_values_20sec.max() - raw_values_20sec.min())
                #normalized_values_20sec = normalized_values_20sec.reshape(original_shape)
                input_20sec_list.append(normalized_values_20sec)
        
        # Forward pass through the autoencoders
        input_10min_batch = torch.tensor(input_10min_list, dtype=torch.float32).to(device)
        input_20sec_batch = torch.tensor(input_20sec_list, dtype=torch.float32).to(device)
        
        if input_10min_batch.shape[0] != batch_size:
            input_10min_batch = F.pad(input_10min_batch, (0, 0, 0, 0, 0, batch_size-input_10min_batch.shape[0]), mode='constant', value=0)
        
        if input_20sec_batch.shape[0] != batch_size:
            input_20sec_batch = F.pad(input_20sec_batch, (0, 0, 0, 0, 0, batch_size-input_20sec_batch.shape[0]), mode='constant', value=0)
        
        output_10min_batch = model_10min(input_10min_batch)
        output_20sec_batch = model_20sec(input_20sec_batch)

        # Calculate loss and perform optimization for 10min autoencoder
        loss_10min = loss_function_10min(output_10min_batch, input_10min_batch)
        optimizer_10min.zero_grad()
        loss_10min.backward()
        optimizer_10min.step()

        # Calculate loss and perform optimization for 20sec autoencoder
        #print(output_20sec_batch.shape, input_20sec_batch.shape)
        loss_20sec = loss_function_20sec(output_20sec_batch, input_20sec_batch)
        
        optimizer_20sec.zero_grad()
        loss_20sec.backward()
        optimizer_20sec.step()

        # Accumulate epoch loss
        epoch_loss_10min += loss_10min.item()
        epoch_loss_20sec += loss_20sec.item()

        # Clean up to avoid memory issues
        del output_10min_batch, output_20sec_batch, input_10min_batch, input_10min_list, input_20sec_batch, input_20sec_list
        
        if i % 20 == 0:
            print(f"Done batch {i}", end = '... ')

    # Calculate average loss for the epoch
    avg_loss_10min = epoch_loss_10min / num_batches
    avg_loss_20sec = epoch_loss_20sec / num_batches

    print(f"Epoch {epoch} Summary: Avg Loss_10min: {avg_loss_10min}, Avg Loss_20sec: {avg_loss_20sec}")
    #print(f"Epoch {epoch} Summary: Avg Loss_10min: {avg_loss_10min}")

Training Autoencoder on 17089 datapoints with batch size 10
Batches 1709: Done batch 0... Done batch 20... Done batch 40... Done batch 60... Done batch 80... Done batch 100... Done batch 120... Done batch 140... Done batch 160... Done batch 180... Done batch 200... Done batch 220... Done batch 240... Done batch 260... Done batch 280... Done batch 300... Done batch 320... Done batch 340... Done batch 360... Done batch 380... Done batch 400... Done batch 420... Done batch 440... Done batch 460... Done batch 480... Done batch 500... Done batch 520... Done batch 540... Done batch 560... Done batch 580... Done batch 600... Done batch 620... Done batch 640... Done batch 660... Done batch 680... Done batch 700... Done batch 720... Done batch 740... Done batch 760... Done batch 780... Done batch 800... Done batch 820... Done batch 840... Done batch 860... Done batch 880... Done batch 900... Done batch 920... Done batch 940... Done batch 960... Done batch 980... Done batch 1000... Done batch 10

In [16]:
%timeit
"""
Get Feature Data 
#We need 4000 features from here to fit into the XGBoost 
Takes about an hour to run
"""
numFeatures_10min = 3600
numFeatures_20sec = 1600
totalFeatures = numFeatures_10min + numFeatures_20sec
#change all the 
print(f"Generating {totalFeatures} features on {len(train)} datapoints")
FEATURES = ["feature_{}_10min".format(i) for i in range(numFeatures_10min)]
FEATURES += ["feature_{}_20sec".format(i) for i in range(numFeatures_20sec)]
data = np.zeros((len(train), len(FEATURES)))

for k in range(len(train)):
    #It is going individual, to make it work for a higher batch size, fix this!!
    if k%100==0: print(k,', ',end='')
    row = train.iloc[k]
    r = int( (row['min'] + row['max'])//4 ) 
    

# 10 MINUTE WINDOW FEATURES
    # this will likey need to be unsqueezed or smth
    raw_data = spectrograms[row.spec_id][r:r+300, :]
#     original_shape = raw_data.shape
    #raw_values_10min = raw_data.flatten()
    normalized_values = (raw_data - raw_data.min()) / (raw_data.max() - raw_data.min())
    x = np.array(model_10min.encoder(torch.tensor([normalized_values]).to(device)).tolist())
    data[k,:numFeatures_10min] = x.flatten() #Can we remove this flattnening?

    # 20 SECOND WINDOW FEATURES 
    # this will likey need to be unsqueezed or smth
    raw_data = spectrograms[row.spec_id][r+145:r+155, :]
#     original_shape = raw_data.shape
    #raw_values_20sec = raw_data.flatten()
    normalized_values = (raw_data - raw_data.min()) / (raw_data.max() - raw_data.min())
    x = np.array(model_20sec.encoder(torch.tensor([normalized_values]).to(device)).tolist())
    data[k,numFeatures_10min:numFeatures_10min+numFeatures_20sec] = x.flatten()
    if k%100==0: print("Done {}".format(k))
print(data.shape)
train[FEATURES] = data #Fails on this memory allocation,we would need to reduce the dimensions of the 10 min data. 
print('New train shape:',train.shape)

Generating 5200 features on 17089 datapoints
0 , Done 0
100 , Done 100
200 , Done 200
300 , Done 300
400 , Done 400
500 , Done 500
600 , Done 600
700 , Done 700
800 , Done 800
900 , Done 900
1000 , Done 1000
1100 , Done 1100
1200 , Done 1200
1300 , Done 1300
1400 , Done 1400
1500 , Done 1500
1600 , Done 1600
1700 , Done 1700
1800 , Done 1800
1900 , Done 1900
2000 , Done 2000
2100 , Done 2100
2200 , Done 2200
2300 , Done 2300
2400 , Done 2400
2500 , Done 2500
2600 , Done 2600
2700 , Done 2700
2800 , Done 2800
2900 , Done 2900
3000 , Done 3000
3100 , Done 3100
3200 , Done 3200
3300 , Done 3300
3400 , Done 3400
3500 , Done 3500
3600 , Done 3600
3700 , Done 3700
3800 , Done 3800
3900 , Done 3900
4000 , Done 4000
4100 , Done 4100
4200 , Done 4200
4300 , Done 4300
4400 , Done 4400
4500 , Done 4500
4600 , Done 4600
4700 , Done 4700
4800 , Done 4800
4900 , Done 4900
5000 , Done 5000
5100 , Done 5100
5200 , Done 5200
5300 , Done 5300
5400 , Done 5400
5500 , Done 5500
5600 , Done 5600
5700 , Don

In [17]:
### To prevent running the encoder part again
# train[FEATURES] = data

# print('New train shape:',train.shape)


In [18]:
print(data.shape)

(17089, 5200)


<b><span style='color:#FFCE30'> 7.2 |</span>  Frequency Band Analysis</b>

#### Frequency Band Feature Extraction:

* The function extract_frequency_band_features is designed to process a segment of EEG data. EEG data is a complex signal that represents the electrical activity of the brain.
* This function divides the EEG signal into different frequency bands: Delta, Theta, Alpha, Beta, and Gamma. These bands are significant in neuroscientific studies as they are associated with different brain states and activities.

![](https://ars.els-cdn.com/content/image/3-s2.0-B9780128044902000026-f02-01-9780128044902.jpg)


1. **Delta (0.5 – 4 Hz):**
Delta waves are the slowest brainwaves and are typically associated with deep sleep and restorative processes in the body. They are most prominent during dreamless sleep and play a role in healing and regeneration.
2. **Theta (4 – 8 Hz):**
Theta waves occur during light sleep, deep meditation, and REM (Rapid Eye Movement) sleep. They are linked to creativity, intuition, daydreaming, and fantasizing. Theta states are often associated with subconscious mind activities.
3. **Alpha (8 – 12 Hz):**
Alpha waves are present during physically and mentally relaxed states but still alert. They are typical in wakeful states that involve a relaxed and effortless alertness. Alpha waves aid in mental coordination, calmness, alertness, and learning.
4. **Beta (12 – 30 Hz):**
Beta waves dominate our normal waking state of consciousness when attention is directed towards cognitive tasks and the outside world. They are associated with active, busy or anxious thinking and active concentration.
5. **Gamma (30 – 45 Hz):**
Gamma waves are involved in higher mental activity and consolidation of information. They are important for learning, memory, and information processing. Gamma waves are thought to be the fastest brainwave frequency and relate to simultaneous processing of information from different brain areas.




* For each frequency band, the function applies a bandpass filter to isolate that band's signal. It then computes statistical features (mean, standard deviation, maximum, and minimum) for each band, effectively capturing the characteristics of the EEG signal in these different frequency ranges.
* The use of np.nanmean, np.nanstd, np.nanmax, and np.nanmin ensures that the calculations are robust to NaN (Not a Number) values in the data, which might occur due to various reasons like signal loss or artifacts.

#### Feature Aggregation and PCA:

* The main script initializes a Principal Component Analysis (PCA) model with the intention of reducing the dimensionality of the extracted features. PCA is a common technique used to transform high-dimensional datasets into a lower-dimensional space while retaining most of the variance in the data.
* The script iterates over rows in the train dataset, extracting EEG segments and applying the extract_frequency_band_features function to each channel in these segments. The extracted features from all channels are aggregated.
* However, before applying PCA, any NaN values in the aggregated data (data_original) are handled using mean imputation. This step ensures that the PCA algorithm, which cannot handle NaN values, receives a clean dataset.
* After imputation, PCA is applied to transform the features into a principal component space, and these transformed features are added back into the train DataFrame.
* This process ultimately results in a feature set that's potentially more informative and concise for machine learning models, helping in tasks like classification or anomaly detection in EEG data.

In [19]:
# print("Introduced band-level features of: ", len(train.columns) - len(FEATURES) - 12)
# print(pca_feature_columns)
# FEATURES += pca_feature_columns
# train.head()

In [20]:
#Scaling the feature values --> scale the features to have a mean of 0 and a standard deviation of 1.
from sklearn.preprocessing import StandardScaler

# Columns to be excluded from scaling
excluded_columns = ['eeg_id', 'spec_id', 'min', 'max', 'patient_id', 'seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote','target']

# Save the columns to be excluded
excluded_data = train[excluded_columns]

# DataFrame with only the columns to be scaled
features = train.drop(columns=excluded_columns)

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler to the features and transform them
features_scaled = scaler.fit_transform(features)

# Create a DataFrame from the scaled features
features_scaled_df = pd.DataFrame(features_scaled, columns=features.columns)

# Concatenate the scaled features with the excluded columns
train_scaled_df = pd.concat([excluded_data.reset_index(drop=True),features_scaled_df,], axis=1)
# train_scaled_df.to_csv("/kaggle/working/")
train_scaled_df 


,eeg_id,spec_id,min,max,patient_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,...,feature_1590_20sec,feature_1591_20sec,feature_1592_20sec,feature_1593_20sec,feature_1594_20sec,feature_1595_20sec,feature_1596_20sec,feature_1597_20sec,feature_1598_20sec,feature_1599_20sec
0,568657,789577333,0.0,16.0,20654,0.0,0.000000,0.25,0.000000,0.166667,...,0.005228,0.057438,0.134112,0.050494,0.140059,0.206816,0.120446,0.184574,-0.094461,0.158359
1,582999,1552638400,0.0,38.0,20230,0.0,0.857143,0.00,0.071429,0.000000,...,0.184596,0.216160,0.263451,0.161152,0.219960,0.128618,0.196309,0.304614,-0.075473,0.166410
2,642382,14960202,1008.0,1032.0,5955,0.0,0.000000,0.00,0.000000,0.000000,...,0.281205,0.316210,0.366085,0.226516,0.312675,0.140743,0.255472,0.306688,-0.065188,0.190639
3,751790,618728447,908.0,908.0,38549,0.0,0.000000,1.00,0.000000,0.000000,...,-0.082966,-0.002932,0.053267,0.077028,0.120787,0.061703,0.055282,0.237846,-0.084412,0.165713
4,778705,52296320,0.0,0.0,40955,0.0,0.000000,0.00,0.000000,0.000000,...,-0.982450,-0.352745,-0.806457,-0.241041,-0.601149,0.343555,-0.135215,-0.040244,-0.172897,-0.128651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17084,4293354003,1188113564,0.0,0.0,16610,0.0,0.000000,0.00,0.000000,0.500000,...,0.305907,0.322216,0.337986,0.214063,0.295522,0.173395,0.211730,0.299751,-0.068270,0.170508
17085,4293843368,1549502620,0.0,0.0,15065,0.0,0.000000,0.00,0.000000,0.500000,...,-1.740965,-1.781943,-0.844376,-0.120955,-0.892295,0.172791,-0.661345,-0.581850,-0.321329,-0.210617
17086,4294455489,2105480289,0.0,0.0,56,0.0,0.000000,0.00,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17087,4294858825,657299228,0.0,12.0,4312,0.0,0.000000,0.00,0.000000,0.066667,...,0.221486,0.095751,0.178685,0.154556,0.209369,0.079292,0.154725,0.113886,-0.097040,0.089446


In [21]:
train_scaled_df.describe()

,eeg_id,spec_id,min,max,patient_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,...,feature_1590_20sec,feature_1591_20sec,feature_1592_20sec,feature_1593_20sec,feature_1594_20sec,feature_1595_20sec,feature_1596_20sec,feature_1597_20sec,feature_1598_20sec,feature_1599_20sec
count,1.708900e+04,1.708900e+04,17089.000000,17089.000000,17089.000000,17089.000000,17089.000000,17089.000000,17089.000000,17089.000000,...,1.700300e+04,1.700300e+04,1.700300e+04,1.700300e+04,1.700300e+04,1.700300e+04,1.700300e+04,1.700300e+04,1.700300e+04,1.700300e+04
mean,2.135226e+09,1.080640e+09,401.650711,431.761191,32839.981977,0.152810,0.142456,0.104062,0.065407,0.114851,...,5.452140e-15,-5.159093e-15,-1.964931e-15,5.501556e-16,4.263131e-15,4.793437e-15,-6.307044e-15,1.181863e-14,-2.075568e-15,5.156376e-15
std,1.235712e+09,6.251739e+08,1226.839779,1232.863269,18351.751174,0.331563,0.295541,0.258825,0.187005,0.271425,...,1.000029e+00,1.000029e+00,1.000029e+00,1.000029e+00,1.000029e+00,1.000029e+00,1.000029e+00,1.000029e+00,1.000029e+00,1.000029e+00
min,5.686570e+05,3.537330e+05,0.000000,0.000000,56.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-9.926762e+00,-1.085007e+01,-1.103980e+01,-7.875155e+00,-2.810939e+01,-1.631628e+01,-9.176712e+00,-1.446819e+01,-2.649211e+00,-1.075000e+01
25%,1.062096e+09,5.396648e+08,0.000000,4.000000,17408.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-5.666393e-02,-3.247033e-02,-1.147676e-02,-1.993139e-02,4.478375e-02,-8.519344e-03,1.361536e-02,5.456708e-02,-1.079614e-01,3.108985e-02
50%,2.123560e+09,1.073264e+09,0.000000,40.000000,32068.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.119973e-01,2.460188e-01,2.611188e-01,1.644766e-01,2.370008e-01,1.414755e-01,1.783483e-01,2.473954e-01,-7.312061e-02,1.627711e-01
75%,3.208261e+09,1.641428e+09,308.000000,346.000000,48272.000000,0.000000,0.068966,0.000000,0.000000,0.000000,...,3.176995e-01,3.578889e-01,3.620840e-01,2.318723e-01,3.024085e-01,1.928465e-01,2.326642e-01,3.061019e-01,-6.266623e-02,1.992190e-01
max,4.294958e+09,2.147388e+09,17556.000000,17632.000000,65494.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,2.166236e+01,2.556704e+01,2.693223e+01,3.117137e+01,2.248262e+01,3.093284e+01,2.796299e+01,3.165761e+01,1.154014e+01,2.028510e+01


# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#FFCE30"><b><span style='color:#FFFFFF'>8 |</span></b> <b>TRAIN MODEL</b></div>

* Original work uses catboost, let's try with XGBoost in this version to see the difference in model performance.

In [22]:
import xgboost as xgb
from sklearn.svm import SVC
import gc
from sklearn.model_selection import KFold, GroupKFold
import pickle
from sklearn.multioutput import MultiOutputRegressor
from scipy.special import rel_entr

In [23]:
len(FEATURES)

5200

In [24]:
## Regressor
%timeit
all_oof = []
all_true = []
TARS = {'Seizure':0, 'LPD':1, 'GPD':2, 'LRDA':3, 'GRDA':4, 'Other':5}
n_splits = 5 
gkf = GroupKFold(n_splits=5) #5
for i, (train_index, valid_index) in enumerate(gkf.split(train_scaled_df, train_scaled_df.target, train_scaled_df.patient_id)):   
    if i >= n_splits:
        continue
    print('#'*25)
    print(f'### Fold {i+1}')
    print(f'### train size {len(train_index)}, valid size {len(valid_index)}')
    print('#'*25)
    
    # Instantiate the XGBRegressor model
    xgb_model = xgb.XGBRegressor(objective='reg:squarederror', learning_rate = 0.1) # uses MSE to predict probabilities

    model = MultiOutputRegressor(xgb_model) # since we have multiple outputs
    
#     model = SVC(probability=True)    
    LABEL_NAMES = ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']
    # Prepare training and validation data
    X_train = train_scaled_df.loc[train_index, FEATURES]
    y_train = train_scaled_df.loc[train_index, LABEL_NAMES]
    X_valid = train_scaled_df.loc[valid_index, FEATURES]
    y_valid = train_scaled_df.loc[valid_index, LABEL_NAMES]
    model.fit(X_train, y_train, verbose=True,) 

    with open(f'XGBoost_f{i}.pkl', 'wb') as f:
        pickle.dump(model, f)

    y_pred = model.predict(X_valid)
    y_pred[y_pred < 0] = 0
    oof = y_pred / np.sum(y_pred, axis=1).reshape(-1,1) # ensure they sum to 1
    true = y_valid.values
    kl_divergence = np.mean(np.sum(true * (np.log(true + 1e-10) - np.log(oof + 1e-10)), axis=1))
    print(f"Kale Divergence: {kl_divergence}")
    
    all_oof.append(oof)
    all_true.append(true)
    
    del X_train, y_train, X_valid, y_valid, oof
    gc.collect()
    
all_oof = np.concatenate(all_oof)
all_true = np.concatenate(all_true)

#########################
### Fold 1
### train size 13671, valid size 3418
#########################
Kale Divergence: 0.9709896458372157
#########################
### Fold 2
### train size 13671, valid size 3418
#########################
Kale Divergence: 0.9897447926359754
#########################
### Fold 3
### train size 13671, valid size 3418
#########################
Kale Divergence: 0.9283146094892828
#########################
### Fold 4
### train size 13671, valid size 3418
#########################
Kale Divergence: 0.9783655268890196
#########################
### Fold 5
### train size 13672, valid size 3417
#########################
Kale Divergence: 1.096386825416861


In [25]:
""" 
With the classifier

"""

# #Reduce the number of splits, otherwise takes very long to run
# all_oof = []
# all_true = []
# TARS = {'Seizure':0, 'LPD':1, 'GPD':2, 'LRDA':3, 'GRDA':4, 'Other':5}
# n_splits = 3
# gkf = GroupKFold(n_splits=3) #5
# for i, (train_index, valid_index) in enumerate(gkf.split(train_scaled_df, train_scaled_df.target, train_scaled_df.patient_id)):   
#     if i >= n_splits:
#         continue
#     print('#'*25)
#     print(f'### Fold {i+1}')
#     print(f'### train size {len(train_index)}, valid size {len(valid_index)}')
#     print('#'*25)
    
#     model = xgb.XGBClassifier(
#         objective='multi:softprob', 
#         num_class=len(TARS),
#         learning_rate = 0.1,                       
#         tree_method='hist',  # skip GPU acceleration
#    )
    
# #     model = SVC(probability=True)    
    
#     # Prepare training and validation data
#     X_train = train_scaled_df.loc[train_index, FEATURES]
#     y_train = train_scaled_df.loc[train_index, 'target'].map(TARS)
#     X_valid = train_scaled_df.loc[valid_index, FEATURES]
#     y_valid = train_scaled_df.loc[valid_index, 'target'].map(TARS)
    
#     model.fit(X_train, y_train, 
#               eval_set=[(X_valid, y_valid)], 
#               verbose=True, 
#               early_stopping_rounds=10)

# #     imputer = SimpleImputer(strategy='mean')
# #     X_train = imputer.fit_transform(X_train)
# #     X_valid = imputer.fit_transform(X_valid)
    
# #     model.fit(X_train, y_train)
#     with open(f'XGBoost_f{i}.pkl', 'wb') as f:
#         pickle.dump(model, f)

#     oof = model.predict_proba(X_valid)
#     true = train_scaled_df.loc[valid_index, TARGETS].values
#     kl_divergence = np.mean(np.sum(true * (np.log(true + 1e-10) - np.log(oof + 1e-10)), axis=1))
#     print(f"KL Divergence: {kl_divergence}")
    
#     all_oof.append(oof)
#     all_true.append(true)
    
#     del X_train, y_train, X_valid, y_valid, oof
#     gc.collect()
    
# all_oof = np.concatenate(all_oof)
# all_true = np.concatenate(all_true)

' \nWith the classifier\n\n'

# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#FFCE30"><b><span style='color:#FFFFFF'>9 |</span></b> <b>HYPERPARAMETER TUNING</b></div>

### <b><span style='color:#FFCE30'> 9.1 |</span> Import Libraries and Set Up Optuna</b>
* First, you import necessary libraries: optuna for hyperparameter optimization, xgboost for the machine learning model, log_loss from scikit-learn for the evaluation metric, and GroupKFold for cross-validation.
* optuna.create_study(direction='minimize') creates a new optimization study. The direction='minimize' means you want to minimize the value returned by the objective function, which in this case is the log loss.

### <b><span style='color:#FFCE30'> 9.2 |</span> Define the Objective Function</b>
* The objective function is what Optuna will optimize. This function takes a trial object, which is used to suggest values for the hyperparameters.
* Inside this function, you set up the hyperparameter space. Optuna will test different combinations of these parameters:
1. lambda, alpha: Regularization parameters.
2. colsample_bytree, subsample: Ratios for column and row sampling.
3. learning_rate: Step size shrinkage used to prevent overfitting.
4. n_estimators: Number of gradient boosted trees.
5. max_depth: Maximum depth of a tree.
6. min_child_weight: Minimum sum of instance weight needed in a child.

### <b><span style='color:#FFCE30'> 9.3 |</span> Cross-Validation Loop</b>

* The function uses GroupKFold for splitting the data. This method is suitable when you have groups in your data (like patient IDs) that should not be split across the training and validation sets.
* For each fold in the cross-validation, the function:
1. Splits the data into training and validation sets.
2. Trains an XGBoost model using the parameters suggested by Optuna.
3. Computes the log loss on the validation set.
4. The average log loss across all folds is returned. Optuna will use this value to decide which hyperparameters are best.

### <b><span style='color:#FFCE30'> 9.4 |</span> Running the Optuna Study</b>

* study.optimize(objective, n_trials=100) tells Optuna to optimize the objective function. It will try 100 different combinations of hyperparameters (n_trials=100) to find the best ones.
* It is best to start with small trials before investing time to run on more trials to manage time invested
* Once the optimization is complete, the best hyperparameters found are printed.

In [26]:
# import optuna
# from sklearn.metrics import log_loss


# def objective(trial):
#     # Hyperparameters to be tuned by Optuna
#     param = {
#         'objective': 'multi:softprob',
#         'num_class': len(TARS),
#         'tree_method': 'gpu_hist',  # use 'gpu_hist' for GPU
#         'lambda': trial.suggest_loguniform('lambda', 1e-4, 10.0),
#         'alpha': trial.suggest_loguniform('alpha', 1e-4, 10.0),
#         'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]),
#         'subsample': trial.suggest_categorical('subsample', [0.6, 0.7, 0.8, 0.9, 1.0]),
#         'learning_rate': trial.suggest_categorical('learning_rate', [0.008, 0.01, 0.02, 0.05, 0.1]),
#         'n_estimators': 1000,
#         'max_depth': trial.suggest_categorical('max_depth', [5, 7, 9, 11, 13]),
#         'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
#     }

#     gkf = GroupKFold(n_splits=5)
#     cv_scores = []

#     for train_index, valid_index in gkf.split(train, train.target, train.patient_id):
#         X_train, X_valid = train.loc[train_index, FEATURES], train.loc[valid_index, FEATURES]
#         y_train, y_valid = train.loc[train_index, 'target'].map(TARS), train.loc[valid_index, 'target'].map(TARS)

#         model = xgb.XGBClassifier(**param)
#         model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False, early_stopping_rounds=10)
#         preds = model.predict_proba(X_valid)
#         cv_scores.append(log_loss(y_valid, preds))

#     return np.mean(cv_scores)

# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=10)  # Increase n_trials for more extensive search

# print('Number of finished trials:', len(study.trials))
# print('Best trial:', study.best_trial.params)

* [I 2024-01-15 06:06:55,585] A new study created in memory with name: no-name-21c5e987-7941-4f05-8e92-2903b9a7e304
* [I 2024-01-15 06:20:36,414] Trial 0 finished with value: 1.0734795198486586 and parameters: {'lambda': 8.578460041884545, 'alpha': 1.0420327876364774, 'colsample_bytree': 1.0, 'subsample': 1.0, 'learning_rate': 0.02, 'max_depth': 7, 'min_child_weight': 30}. Best is trial 0 with value: 1.0734795198486586.
* [I 2024-01-15 06:39:35,894] Trial 1 finished with value: 1.0792154540612213 and parameters: {'lambda': 0.0029055528938438085, 'alpha': 0.03244985452963714, 'colsample_bytree': 0.5, 'subsample': 1.0, 'learning_rate': 0.01, 'max_depth': 11, 'min_child_weight': 77}. Best is trial 0 with value: 1.0734795198486586.
* [I 2024-01-15 06:42:19,332] Trial 2 finished with value: 1.107279543251576 and parameters: {'lambda': 1.5919274718287213, 'alpha': 0.042136459342788604, 'colsample_bytree': 0.7, 'subsample': 0.7, 'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 152}. Best is trial 0 with value: 1.0734795198486586.
* [I 2024-01-15 06:57:56,577] Trial 3 finished with value: 1.107956784549986 and parameters: {'lambda': 6.873357111288177, 'alpha': 0.05538406375764404, 'colsample_bytree': 0.5, 'subsample': 0.8, 'learning_rate': 0.008, 'max_depth': 7, 'min_child_weight': 108}. Best is trial 0 with value: 1.0734795198486586.
* [I 2024-01-15 07:12:36,921] Trial 4 finished with value: 1.1453593669298952 and parameters: {'lambda': 0.0012348624625841455, 'alpha': 6.698933350539058, 'colsample_bytree': 0.8, 'subsample': 0.9, 'learning_rate': 0.01, 'max_depth': 9, 'min_child_weight': 258}. Best is trial 0 with value: 1.0734795198486586.
* [I 2024-01-15 07:27:45,543] Trial 5 finished with value: 1.1234561427497631 and parameters: {'lambda': 0.0779496745099949, 'alpha': 0.001997110519034328, 'colsample_bytree': 0.5, 'subsample': 0.7, 'learning_rate': 0.008, 'max_depth': 11, 'min_child_weight': 145}. Best is trial 0 with value: 1.0734795198486586.
* [I 2024-01-15 07:30:45,104] Trial 6 finished with value: 1.139466297500727 and parameters: {'lambda': 0.011643281906929, 'alpha': 0.06334100511005662, 'colsample_bytree': 0.6, 'subsample': 0.7, 'learning_rate': 0.1, 'max_depth': 9, 'min_child_weight': 274}. Best is trial 0 with value: 1.0734795198486586.
* [I 2024-01-15 07:33:47,478] Trial 7 finished with value: 1.074140292040741 and parameters: {'lambda': 5.487810272015954, 'alpha': 2.266845998962579, 'colsample_bytree': 0.6, 'subsample': 0.8, 'learning_rate': 0.1, 'max_depth': 7, 'min_child_weight': 51}. Best is trial 0 with value: 1.0734795198486586.
* [I 2024-01-15 07:48:19,863] Trial 8 finished with value: 1.1705891200455967 and parameters: {'lambda': 0.03333036846711228, 'alpha': 0.0004482362892025373, 'colsample_bytree': 0.9, 'subsample': 0.8, 'learning_rate': 0.008, 'max_depth': 13, 'min_child_weight': 294}. Best is trial 0 with value: 1.0734795198486586.
* [I 2024-01-15 07:53:41,442] Trial 9 finished with value: 1.115599617296683 and parameters: {'lambda': 0.000284724944614318, 'alpha': 0.020480207664040264, 'colsample_bytree': 0.6, 'subsample': 0.9, 'learning_rate': 0.05, 'max_depth': 9, 'min_child_weight': 248}. Best is trial 0 with value: 1.0734795198486586.
Number of finished trials: 10

* **Best trial: {'lambda': 8.578460041884545, 'alpha': 1.0420327876364774, 'colsample_bytree': 1.0, 'subsample': 1.0, 'learning_rate': 0.02, 'max_depth': 7, 'min_child_weight': 30}**

# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#FFCE30"><b><span style='color:#FFFFFF'>10 |</span></b> <b>FEATURE IMPORTANCE</b></div>

In [27]:
# TOP = 30

# # better to load model here...

# # Assuming 'model' is your trained model
# feature_importance = model.feature_importances_

# # Get the feature names from 'train'
# feature_names = train.columns

# # Sort the feature importances and get the indices of the sorted array
# sorted_idx = np.argsort(feature_importance)

# # Plot only the top 'TOP' features
# fig = plt.figure(figsize=(10, 8))
# plt.barh(np.arange(len(sorted_idx))[-TOP:], feature_importance[sorted_idx][-TOP:], align='center')
# plt.yticks(np.arange(len(sorted_idx))[-TOP:], feature_names[sorted_idx][-TOP:])
# plt.title(f'Feature Importance - Top {TOP}')
# plt.show()

# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#FFCE30"><b><span style='color:#FFFFFF'>11 |</span></b> <b>INFER TEST</b></div>

In [28]:
test = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/test.csv')
print('Test shape',test.shape)
test.head()

Test shape (1, 3)


,spectrogram_id,eeg_id,patient_id
0,853520,3911565283,6885


In [29]:
%%time
# READ ALL TEST SPECTROGRAMS
PATH2 = '/kaggle/input/hms-harmful-brain-activity-classification/test_spectrograms/'
files = os.listdir(PATH2)
print(f'There are {len(files)} spectrogram parquets')

spectrograms_test = {}
for i,f in enumerate(files):
    if i%100==0: print(i,', ',end='')
    tmp = pd.read_parquet(f'{PATH2}{f}')
    name = int(f.split('.')[0])
    spectrograms_test[name] = tmp.iloc[:,1:].values


There are 1 spectrogram parquets
0 , CPU times: user 41.6 ms, sys: 4.99 ms, total: 46.6 ms
Wall time: 47.3 ms


In [30]:
%time
# ENGINEER FEATURES
import warnings
warnings.filterwarnings('ignore')

SPEC_COLS = pd.read_parquet(f'{PATH}1000086677.parquet').columns[1:]

TEST_FEATURES = FEATURES

print(f'We are creating {len(TEST_FEATURES)} features for {len(test)} rows... ',end='')


# A data matrix data is initialized to store the new features for each eeg_id in the train DataFrame.
# For each row in train, the code calculates the mean and minimum values within the specified 10-minute and 20-second windows.
# These calculated values are then stored in the data matrix.
# Finally, the matrix is added to the train DataFrame as new columns.

data_test = np.zeros((len(test),len(TEST_FEATURES)))
for k in range(len(test)):
    if k%100==0: print(k,', ',end='')
    row = test.iloc[k]       
    s = int( row.spectrogram_id )
    spec = pd.read_parquet(f'{PATH2}{s}.parquet')
    #original_shape = spec.iloc[:,1:].shape
    #raw_values_10min = np.log(spec.iloc[:,1:].values.flatten() + log_noise)
    #raw_values_10min = np.log(spec.iloc[:,1:] + log_noise)
    raw_values_10min = spec.iloc[:,1:].values
    normalized_values = (raw_values_10min - raw_values_10min.min()) / (raw_values_10min.max() - raw_values_10min.min())
    x = np.array(model_10min.encoder(torch.tensor([normalized_values]).to(device)).tolist())   
    data_test[k,:numFeatures_10min] = x.flatten()

    # 20 SECOND WINDOW FEATURES 
    # this will likey need to be unsqueezed or smth
    # original_shape= spec.iloc[145:155,1:].shape
    # raw_values_20sec = np.log(spec.iloc[145:155,1:].values.flatten() + log_noise)
    #raw_values_20sec = np.log(spec.iloc[145:155,1:] + log_noise)
    raw_values_20sec = spec.iloc[145:155,1:].values
    normalized_values = (raw_values_20sec - raw_values_20sec.min()) / (raw_values_20sec.max() - raw_values_20sec.min())
    x = np.array(model_20sec.encoder(torch.tensor([normalized_values]).to(device)).tolist())  
    data_test[k,numFeatures_10min:numFeatures_10min+numFeatures_20sec] = x.flatten()

print(len(TEST_FEATURES), data_test.shape)
test[TEST_FEATURES] = data_test

    

print('New test shape:',test.shape)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.63 µs
We are creating 5200 features for 1 rows... 0 , 5200 (1, 5200)
New test shape: (1, 5203)


In [31]:

# Columns to be excluded from scaling
excluded_columns = ['eeg_id', 'spectrogram_id', 'patient_id']

# Save the columns to be excluded
excluded_data = test[excluded_columns]

# DataFrame with only the columns to be scaled
features = test.drop(columns=excluded_columns)

# Initialize the StandardScaler
# scaler = StandardScaler()

# Fit the scaler to the features and transform them
features_scaled = scaler.transform(features)

# Create a DataFrame from the scaled features
features_scaled_df = pd.DataFrame(features_scaled, columns=features.columns)

# Concatenate the scaled features with the excluded columns
test_scaled_df = pd.concat([excluded_data.reset_index(drop=True),features_scaled_df,], axis=1)
test_scaled_df

,eeg_id,spectrogram_id,patient_id,feature_0_10min,feature_1_10min,feature_2_10min,feature_3_10min,feature_4_10min,feature_5_10min,feature_6_10min,...,feature_1590_20sec,feature_1591_20sec,feature_1592_20sec,feature_1593_20sec,feature_1594_20sec,feature_1595_20sec,feature_1596_20sec,feature_1597_20sec,feature_1598_20sec,feature_1599_20sec
0,3911565283,853520,6885,0.655698,0.163412,0.322061,0.22171,0.152669,0.549196,0.651799,...,0.3338,0.364002,0.367583,0.230298,0.289591,0.198333,0.235506,0.299616,-0.066171,0.201747


In [32]:
# INFER XGBOOST ON TEST
preds = []

for i in range(n_splits):
    print(i, ', ', end='')
    
    # Load the XGBoost model
    with open(f'XGBoost_f{i}.pkl', 'rb') as f:
        model = pickle.load(f)
    
    # Make predictions
    test_data_scaled = test_scaled_df[TEST_FEATURES]
    
    # data_imputed = imputer.fit_transform(test_data_scaled)
    
    pred = model.predict(test_data_scaled)
    pred[pred < 0] = 0
    pred = pred / np.sum(pred, axis=1).reshape(-1,1)
    preds.append(pred) 

# Average the predictions from each fold
pred = np.mean(preds, axis=0)
print('Test preds shape', pred.shape)

0 , 1 , 2 , 3 , 4 , Test preds shape (1, 6)


# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#FFCE30"><b><span style='color:#FFFFFF'>12 |</span></b> <b>SUBMISSION</b></div>

In [33]:
sub = pd.DataFrame({'eeg_id':test.eeg_id.values})
sub[TARGETS] = pred
sub.to_csv('submission.csv',index=False)
print('Submission shape',sub.shape)
sub.head()

Submission shape (1, 7)


,eeg_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,3911565283,0.156608,0.095895,0.000063,0.063611,0.150019,0.533804


In [34]:
# SANITY CHECK TO CONFIRM PREDICTIONS SUM TO ONE
sub.iloc[:,-6:].sum(axis=1)

0    1.0
dtype: float32